In [1]:
import os
from fnmatch import fnmatch
import numpy as np
from PIL import Image
from filtering.filters import Median
from feature_extraction.lbp import Lbp
import sklearn
import time
from gmm import Gmm
from skimage.feature import local_binary_pattern
from scipy.stats import itemfreq
from sklearn.preprocessing import normalize
import pickle

In [2]:
#Read features of training samples (30000x26)
print("loading features")
train_samples_feats_matrix_LBP = np.load("train_samples_feats_matrix_LBP.npy")
print("features loaded")

loading features
features loaded


In [5]:
print (train_samples_feats_matrix_LBP)

[[ 0.08447266  0.01733398  0.02392578 ...,  0.01757812  0.05786133
   0.51708984]
 [ 0.0871582   0.01806641  0.02587891 ...,  0.01806641  0.04956055
   0.55541992]
 [ 0.07397461  0.02441406  0.02270508 ...,  0.01538086  0.05371094
   0.5065918 ]
 ..., 
 [ 0.09277344  0.01635742  0.01977539 ...,  0.01635742  0.04370117
   0.53759766]
 [ 0.08544922  0.01879883  0.02368164 ...,  0.0144043   0.05126953
   0.52270508]
 [ 0.08349609  0.02270508  0.02441406 ...,  0.02124023  0.05004883
   0.59887695]]


In [ ]:
#Train a gaussian mixture model from training data
t0 = time.time()
gm_model = sklearn.mixture.GaussianMixture(n_components=300, covariance_type='full')
gm_model.fit(train_samples_feats_matrix_LBP)
print("traing time: ", time.time() - t0)

In [ ]:
#load validation samples feature matrix (10000x26)
valid_samples_feats_matrix = np.load("valid_samples_feats_matrix_LBP.npy")

In [2]:
test_samples_feats_matrix = np.load("test_samples_feats_matrix_LBP.npy")

In [3]:
gm_model = pickle.load( open( 'GMM_LBP_500', "rb" ))

In [4]:
#Predict labels of the new data points
y_predict = gm_model.predict(test_samples_feats_matrix)
#Predict novelty score of new data points
y_predict_score = gm_model.score_samples(test_samples_feats_matrix)
tmp = []

#x_predict_score = gm_model.score_samples(train_samples_feats_matrix_LBP)
#novelty_thresh = np.amin(x_predict_score)

# if data point belongs to the model then 1 else 0
tmp = np.repeat(1, 10000)
tmp[y_predict_score < 152.270658392] = 0


In [ ]:
print novelty_thresh

In [5]:
#Ground truth
y1 = np.repeat(1, 6000) #normal
y2 = np.repeat(0, 4000) #abnormal
y = np.concatenate((y1,y2))

In [6]:
f1_binary = sklearn.metrics.f1_score(y, tmp,pos_label = 0, average = 'binary')
f1_macro = sklearn.metrics.f1_score(y, tmp, average = 'macro')
auc = sklearn.metrics.roc_auc_score(y, y_predict_score)
Math_Cof = sklearn.metrics.matthews_corrcoef(y, tmp) 
print ("f1_binary: ", f1_binary, "f1_macro: ", f1_macro, "MAth_cof: ", Math_Cof , " auc: ", auc)

('f1_binary: ', 0.5737184115523466, 'f1_macro: ', 0.30685920577617332, 'MAth_cof: ', 0.05541719940458658, ' auc: ', 0.60588525000000004)


In [ ]:
#Get different thresholds value from ROC with corresponding F1_score and AUC
fpr,tpr,thresh = sklearn.metrics.roc_curve(y, y_predict_score)
for t, thres in enumerate (thresh) :
    tmp = np.repeat(0,10000)
    tmp[y_predict_score < thres] = 1
    f1_binary = sklearn.metrics.f1_score(y, tmp, average = 'binary')
    f1_macro = sklearn.metrics.f1_score(y, tmp, average = 'macro')
    Math_Cof = sklearn.metrics.matthews_corrcoef(y, tmp) 
    print ("f1_binary: ", f1_binary, "f1_macro: ", f1_macro, "MAth_cof: ", Math_Cof , "thres: ", thres)

In [ ]:
import matplotlib.pyplot as pl
import matplotlib
from skimage import io

fpr,tpr,thresh = sklearn.metrics.roc_curve(y, y_predict_score)

f,ax = pl.subplots(1,1)
ax.plot(fpr,tpr,label="GMM")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC curves")
ax.legend(loc="lower right")
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='square', facecolor='white', alpha=0.5)

textstr = '$F1-binary=%.2f$\n$F1-macro=%.2f$\n$math-corcoeffient=%.2f$\n$AUC=%.2f$'%(0.58, 0.64, 0.29, 0.69)
# place a text box in upper left in axes coords
ax.text(0.95, 0.2, textstr, transform=ax.transAxes, fontsize=14,
    horizontalalignment='right', verticalalignment='bottom', bbox=props)

io.show()

In [ ]:
f.savefig('GMM_LBP_500.png')